In [ ]:
# !pip install transformers

## Summazition

In [2]:
# Import the pipeline class from the transformers module. 
from transformers import pipeline
# Instantiate the pipeline class for summarization using the facebook/bart-large-cnn model.
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/opt/anaconda3/envs/tensorflow/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/tensorflow/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Device set to use mps:0


In [4]:
# Create a variable to contain the text from (https://en.wikipedia.org/wiki/Deep_learning) to summarize.
article ="""Deep learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.[2] 

Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]

Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, artificial neural networks tend to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) and analog.[6][7]

The adjective "deep" in deep learning refers to the use of multiple layers in the network. Early work showed that a linear perceptron cannot be a universal classifier, but that a network with a nonpolynomial activation function with one hidden layer of unbounded width can. Deep learning is a modern variation that is concerned with an unbounded number of layers of bounded size, which permits practical application and optimized implementation, while retaining theoretical universality under mild conditions. In deep learning the layers are also permitted to be heterogeneous and to deviate widely from biologically informed connectionist models, for the sake of efficiency, trainability and understandability."""

In [5]:
# Get the most likely summary of the article using "False" for the `do_sample` parameter.
most_likely_summary = summarizer(article, 
                     min_length=30, 
                     max_length=130, 
                     do_sample=False)

# Display the summary
most_likely_summary

[{'summary_text': 'Deep learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design and medical image analysis.'}]

In [6]:
# Get the summary text from the JSON output
most_likely_summary[0]["summary_text"]

'Deep learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design and medical image analysis.'

In [7]:
# Get a more diverse summary of the article using "True" for the `do_sample` parameter.
diverse_summary = summarizer(article, 
                     min_length=30, 
                     max_length=130, 
                     do_sample=True)[0]["summary_text"]

# Display the summary
diverse_summary

'Deep learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation and bioinformatics.'

## Question and answer

In [ ]:
# pip install transformers
# pip install gradio

In [8]:
# Import transformers pipeline
from transformers import pipeline
# Import Gradio
import gradio as gr

In [9]:
# Initialize the pipeline to generate questions and answers using the distilbert-base-cased-distilled-squad model. 
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

Device set to use mps:0


In [11]:
# Create a function called `question_answer` that takes two parameters, the text to search and a question.
# The function should return the question, answer, probability score, and the starting and ending index of the answer.
def question_answer(text, question):
    result = question_answerer(question=question, context=text)
    return question, result['answer'], result['score'], result['start'], result['end']

In [12]:
# Create the app with two Textbox components. 
# The first textbox will take the text to search the second will take the question.
# The output should show the question, answer, probability score, and the starting and ending index of the answer.

app = gr.Interface(
    fn=question_answer,
    inputs = [
        gr.Textbox(label="Paste the text to search."), 
        gr.Textbox(label="Ask a question.")],
    outputs=gr.Textbox(lines=10, label="Answer to question, probability score, and location.", show_copy_button=True))
    
# Launch the app.
app.launch(show_error=True)

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------


## Upload Document

In [1]:
def extract_text_from_file(file_path):
    """Extract text from PDF, Word, or plain text files."""
    if file_path.endswith('.pdf'):
        with open(file_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        return text
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
        return text
    elif file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    else:
        raise ValueError("Unsupported file format. Please upload a PDF, Word (.docx), or plain text (.txt) file.")